In [6]:
#!pip install svgwrite
import pandas as pd
import svgwrite

In [7]:
# Load the data
data = pd.read_csv('spells.csv')

# Convert the data to long format
df_melted = pd.melt(data, id_vars=['name', 'level', 'school'], 
                    value_vars=['bard', 'cleric', 'druid', 'paladin', 'ranger', 'sorcerer', 'warlock', 'wizard'], 
                    var_name='class', value_name='has_spell')

# Group by grade, level, school and count the number TRUE
spell_classes = df_melted[df_melted['has_spell'] == True].groupby(['class', 'level', 'school']).size().\
    reset_index(name='spell_count')

spell_classes['school'] = spell_classes['school'].str.strip().str.capitalize() # Normalize titles of schools

In [8]:
# Define parameters for drawing
sizew, sizeh = 25, 25  
marginw, marginh = (sizew /4), (sizeh / 4) # Indents between rectangles

# Indents between schools of magic - they will be grouped in the picture
sch_marg = marginw*4 
pic_h = str((sizeh + marginh) * 10)+"px" # Determine the height of the image based on the size of the tiles

# Maximum number of spells per school
school_max_values = spell_classes.groupby('school')['spell_count'].max().to_dict()  


In [9]:
# There is no official palette, so I picked up the colors by refining the found on the network
school_colors = {
    'Abjuration': '#FFC53A', 
    'Conjuration': '#084B83', 
    'Divination': '#93A8AC',
    'Enchantment': '#21FA90', 
    'Evocation': '#DE3C4B', 
    'Illusion': '#AA7DCE',
    'Necromancy': '#230903', 
    'Transmutation': '#AD9F71'
} 

In [13]:
# Iterate by class
for class_name, class_data in spell_classes.groupby('class'):
    file_ = f"images/{class_name}_data.svg"  # The name of the file for each class

    # Create an SVG object
    dwg = svgwrite.Drawing(file_, profile='tiny', size=('2029px', pic_h))

    # Initial coordinates for drawing
    startx, starty = 0, 0
    x_pos, y_pos = startx, starty

    # Iterate by school
    for column in sorted(class_data['school'].unique()):
        color = school_colors.get(column, 'gray') 
        
        # Maximum number of spells for each school
        max_spells_for_spacing = school_max_values.get(column, 0)  

        # Iterate through the levels
        for level in range(0, class_data['level'].max() + 1):
            spells_in_level = class_data[(class_data['level'] == level) & (class_data['school'] == column)]
            
            # Sum the number of spells for level and school
            value = spells_in_level['spell_count'].sum()  

            for n in range(value):
                dwg.add(dwg.rect(insert=(x_pos, y_pos), size=(sizew, sizeh), fill=color))
                x_pos += sizew + marginw  # Shift to the right

            y_pos += sizeh + marginh  # Slide down for the next level
            x_pos = startx  # Return to the initial X-axis position

        y_pos = starty  # Return to the initial Y-axis position
        startx += max_spells_for_spacing * (sizew + marginw) + marginw + sch_marg  # Shift to the right for the new school
        x_pos = startx  # Update the initial x-coordinate

    # Save the file
    dwg.save()
    print(f"SVG for {class_name} saved as {file_}")

SVG for bard saved as images/bard_data.svg
SVG for cleric saved as images/cleric_data.svg
SVG for druid saved as images/druid_data.svg
SVG for paladin saved as images/paladin_data.svg
SVG for ranger saved as images/ranger_data.svg
SVG for sorcerer saved as images/sorcerer_data.svg
SVG for warlock saved as images/warlock_data.svg
SVG for wizard saved as images/wizard_data.svg
